In [1]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.externals import joblib

from sklearn.utils import shuffle
sys.path.append( os.getcwd()+'/modules' )
import aux_functions

In [16]:
#Loading datasets
path = os.getcwd() 
os.chdir( "/home/toful/Documents/DataSets/cresci-2017.csv/datasets_full.csv/" )

real = pd.read_csv( 'genuine_accounts.csv/genuine_accounts.csv/users.csv' )
real = real.fillna( '' )
real['knownbot'] = 0

fakeFollowers = pd.read_csv('fake_followers.csv/fake_followers.csv/users.csv', na_filter=False)
fakeFollowers = fakeFollowers.fillna( '' )
fakeFollowers['knownbot'] = 1

socialSpamBots1 = pd.read_csv('social_spambots_1.csv/social_spambots_1.csv/users.csv', na_filter=False)
socialSpamBots1 = socialSpamBots1.fillna( '' )
socialSpamBots1['knownbot'] = 1

socialSpamBots2 = pd.read_csv('social_spambots_2.csv/social_spambots_2.csv/users.csv', na_filter=False)
socialSpamBots2 = socialSpamBots2.fillna( '' )
socialSpamBots2['knownbot'] = 1

socialSpamBots3 = pd.read_csv('social_spambots_3.csv/social_spambots_3.csv/users.csv', na_filter=False)
socialSpamBots3 = socialSpamBots3.fillna( '' )
socialSpamBots3['knownbot'] = 1

df_list = []

df_list += [ shuffle( pd.concat( [ real, fakeFollowers ], sort=False ) ) ]
df_list += [ shuffle( pd.concat( [ real, socialSpamBots1, socialSpamBots2, socialSpamBots3 ], sort=False ) ) ]
df_list += [ shuffle( pd.concat( [ real, fakeFollowers, socialSpamBots1, socialSpamBots2, socialSpamBots3 ], sort=False ) ) ]

#df = shuffle( df )
#print df.head()
#print df.info()

os.chdir( path )


In [27]:
print( os.getcwd()  )

/home/toful/Documents/DataSets/cresci-2017.csv/datasets_full.csv


In [17]:
#Building the working datasets
score_list = []
for df in df_list:
    score = pd.DataFrame()
    score['id'] = df['id']

    score['lang-en'] = df.apply( lambda row: aux_functions.language (row), axis=1 )
    score['profile_pic'] = df.apply( lambda row: aux_functions.profile_image (row), axis=1 ) #check this feature
    score['def_profile_pic'] = df.apply( lambda row: aux_functions.def_profile_image (row), axis=1 )
    score['has_screen_name'] = df.apply( lambda row: aux_functions.screen_name (row), axis=1 )
    score['30followers'] = df.apply( lambda row: aux_functions.min_followers (row, 30), axis=1 )
    score['1000friends'] = df.apply( lambda row: aux_functions.min_friends (row, 1000), axis=1 )
    score['twice_num_followers'] = df.apply( lambda row: aux_functions.ratio_followers (row), axis=1 )
    score['fifty_FriendsFollowersRatio'] = df.apply( lambda row: aux_functions.ratio_followers2 (row, 50), axis=1 )
    score['hundred_FriendsFollowersRatio'] = df.apply( lambda row: aux_functions.ratio_followers2 (row, 100), axis=1 )
    score['geoloc'] = df.apply( lambda row: aux_functions.location (row), axis=1 )
    score['banner_link'] = df.apply( lambda row: aux_functions.profile_banner (row), axis=1 )
    score['50tweets'] = df.apply( lambda row: aux_functions.tweets_written (row, 50), axis=1 )
    score['20statuses'] = df.apply( lambda row: aux_functions.min_statuses (row, 20), axis=1 )
    score['NeverTweeted'] = df.apply( lambda row: aux_functions.never_tweeted (row ), axis=1 )
    score['has_description'] = df.apply( lambda row: aux_functions.description (row), axis=1 )
    score['knownbot'] = df.apply( lambda row: aux_functions.knownbot (row), axis=1 )

    print score.shape
    print score.describe()
    print score.columns.values
    score_list += [ score ]
    #score

(6825, 17)
                 id      lang-en  profile_pic  def_profile_pic  \
count  6.825000e+03  6825.000000       6825.0      6825.000000   
mean   8.499737e+08     0.898022          0.0         0.002784   
std    7.662248e+08     0.302642          0.0         0.052693   
min    6.780330e+05     0.000000          0.0         0.000000   
25%    2.597067e+08     1.000000          0.0         0.000000   
50%    6.168972e+08     1.000000          0.0         0.000000   
75%    1.174963e+09     1.000000          0.0         0.000000   
max    3.164942e+09     1.000000          0.0         1.000000   

       has_screen_name  30followers  1000friends  twice_num_followers  \
count      6825.000000  6825.000000  6825.000000          6825.000000   
mean          0.000147     0.492601     0.076337             0.588425   
std           0.012105     0.499982     0.265556             0.492155   
min           0.000000     0.000000     0.000000             0.000000   
25%           0.000000     0.

# Data Analisis

In [19]:
from sklearn.model_selection import ShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics as mt

In [18]:
#Preparing the X and Y values for the machine learning analisis
yy=[]
XX=[]
for score in score_list:
    if 'knownbot' in score:
        y = score['knownbot'].values # get the labels we want
        X = score
        del X['knownbot']
        del X['id']
        X = X.values # use everything else to predict
        yy += [y]
        XX += [X]

In [25]:
def train_model( X, y, name ):
    num_cv_iterations = 3
    num_instances = len(y)
    cv_object = ShuffleSplit( n_splits=num_cv_iterations, test_size = 0.2 )

    # first we create a reusable logisitic regression and random forest objects
    lr_clf = LogisticRegression( penalty='l2', C=1.0, class_weight=None ) # get object
    rf_clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0)

    iter_num=0
    # the indices are the rows used for training and testing in each iteration
    for train_indices, test_indices in cv_object.split( X, y ):
        X_train = X[train_indices]
        y_train = y[train_indices]
        X_test = X[test_indices]
        y_test = y[test_indices]

        # train the reusable logisitc regression model on the training data
        #lr_clf.fit( X_train, y_train ) # train object
        #y_pred = lr_clf.predict( X_test ) # get test set precitions

        rf_clf.fit( X_train, y_train ) # train object
        y_pred = rf_clf.predict( X_test ) # get test set precitions
        acc = mt.accuracy_score( y_test, y_pred )
        conf = mt.confusion_matrix( y_test, y_pred )
        # now let's get the accuracy and confusion matrix for this iterations of training/testing
        print "\n====Iteration",iter_num," ===="
        print "RANDOM FOREST"
        print "accuracy: ", acc
        print "confusion matrix\n", conf
        tp = float( conf[1][1] )
        fp = float( conf[0][1] )
        fn = float( conf[1][0] )
        precission = tp / (tp + fp)
        recall = tp / (tp + fn)
        f1 = 2*precission*recall/(precission+recall)
        print "F1: ", f1, "\tPrecission: ", precission, "\tRecall: ", recall
        iter_num+=1
    # save the model to disk
    if not os.path.exists( 'models' ):
        os.makedirs( 'models' )
    str_ = 'models/randomForest_' + name + '_model.sav'
    print( "Saving the model into the disk" )
    joblib.dump( rf_clf, str_ )

In [26]:
names = [ 'fakeFollowers', 'spamBots', 'mix' ]
for i in range( len( names ) ):
    print( "Creating a model to detect:" + names[i] )
    train_model( XX[i], yy[i], names[i] )

Creating a model to detect:fakeFollowers

====Iteration 0  ====
RANDOM FOREST
accuracy:  0.9641025641025641
confusion matrix
[[692  15]
 [ 34 624]]
F1:  0.962220508867 	Precission:  0.976525821596 	Recall:  0.948328267477

====Iteration 1  ====
RANDOM FOREST
accuracy:  0.9692307692307692
confusion matrix
[[705  18]
 [ 24 618]]
F1:  0.967136150235 	Precission:  0.971698113208 	Recall:  0.96261682243

====Iteration 2  ====
RANDOM FOREST
accuracy:  0.9714285714285714
confusion matrix
[[677  12]
 [ 27 649]]
F1:  0.970830216904 	Precission:  0.981845688351 	Recall:  0.960059171598
Saving the model into the disk
Creating a model to detect:spamBots

====Iteration 0  ====
RANDOM FOREST
accuracy:  0.935041716328963
confusion matrix
[[630  68]
 [ 41 939]]
F1:  0.94514343231 	Precission:  0.932472691162 	Recall:  0.958163265306

====Iteration 1  ====
RANDOM FOREST
accuracy:  0.9445768772348033
confusion matrix
[[665  61]
 [ 32 920]]
F1:  0.951888256596 	Precission:  0.937818552497 	Recall:  0.966

In [8]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

1.13.1


In [36]:
#Setting up the model layers
#https://www.tensorflow.org/api_docs/python/tf/keras/layers
#https://www.tensorflow.org/tutorials/keras/basic_classification

model = keras.Sequential( [ keras.layers.Dense( 32, input_shape=( len(X[0]), ) ),
                            keras.layers.Dense( 40, activation = tf.nn.sigmoid ),
                            keras.layers.Dense( 10, activation = tf.nn.sigmoid ),
                            keras.layers.Dense( 2, activation = tf.nn.softmax ) ] )

#the optimazer has to minimize the loss function

#Compiling the model
  #Loss function — This measures how accurate the model is during training. We want to minimize 
                   #this function to "steer" the model in the right direction.
  #Optimizer — This is how the model is updated based on the data it sees and its loss function.
  #Metrics —  Used to monitor the training and testing steps. The following example uses accuracy,
              #the fraction of the images that are correctly classified.
model.compile( optimizer = keras.optimizers.Adam(lr=0.001), loss = 'mean_squared_error', metrics = ['accuracy'] )
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

num_cv_iterations = 1
cv_object = ShuffleSplit( n_splits=num_cv_iterations, test_size = 0.2 )
iter_num = 1

for train_indices, test_indices in cv_object.split( X, y ):
    X_train = X[train_indices]
    y_train = y[train_indices]
    X_test = X[test_indices]
    y_test = y[test_indices]
    
    print "\n\n====Iteration",iter_num," ===="
    
    model.fit( X_train, y_train, epochs=1000 )
    test_loss, test_acc = model.evaluate( X_test, y_test )
    y_pred = model.predict( X_test )
    aux=[]
    for i in range( len( y_pred ) ):
        aux += [ np.argmax( y_pred[i] ) ]
    conf = mt.confusion_matrix( y_test, aux )
    
    print '\nMODEL RESULTS\n\tTest accuracy: ', test_acc, 'Test loss: ', test_loss
    print "PREDICTION RESULTS"
    print "confusion matrix\n", conf
    tp = float( conf[1][1] )
    fp = float( conf[0][1] )
    fn = float( conf[1][0] )
    try:  
        precission = tp / (tp + fp)
        recall = tp / (tp + fn)
        f1 = 2*precission*recall/(precission+recall)
    except ArithmeticError:  
        print ("Arithmetic exception raised." )
    else: 
        print "\tF1: ", f1, "\tPrecission: ", precission, "\tRecall: ", recall
    iter_num+=1

#print y_pred



====Iteration 1  ====
Epoch 1/1000
5460/5460 [==============================] - 1s 150us/sample - loss: 0.2506 - acc: 0.4150
Epoch 2/1000
5460/5460 [==============================] - 0s 71us/sample - loss: 0.2500 - acc: 0.3846
Epoch 3/1000
5460/5460 [==============================] - 0s 68us/sample - loss: 0.2500 - acc: 0.4275
Epoch 4/1000
5460/5460 [==============================] - 0s 79us/sample - loss: 0.2500 - acc: 0.4183
Epoch 5/1000
5460/5460 [==============================] - 1s 136us/sample - loss: 0.2500 - acc: 0.4456
Epoch 6/1000
5460/5460 [==============================] - 1s 99us/sample - loss: 0.2500 - acc: 0.5568
Epoch 7/1000
5460/5460 [==============================] - 0s 66us/sample - loss: 0.2500 - acc: 0.5454
Epoch 8/1000
5460/5460 [==============================] - 0s 69us/sample - loss: 0.2500 - acc: 0.5744
Epoch 9/1000
5460/5460 [==============================] - 0s 67us/sample - loss: 0.2500 - acc: 0.5881
Epoch 10/1000
5460/5460 [==============================]

5460/5460 [==============================] - 0s 69us/sample - loss: 0.2500 - acc: 0.5038
Epoch 160/1000
5460/5460 [==============================] - 0s 66us/sample - loss: 0.2500 - acc: 0.4987
Epoch 161/1000
5460/5460 [==============================] - 0s 66us/sample - loss: 0.2500 - acc: 0.5020
Epoch 162/1000
5460/5460 [==============================] - 0s 67us/sample - loss: 0.2500 - acc: 0.5022
Epoch 163/1000
5460/5460 [==============================] - 0s 67us/sample - loss: 0.2500 - acc: 0.4998
Epoch 164/1000
5460/5460 [==============================] - 0s 65us/sample - loss: 0.2500 - acc: 0.5004
Epoch 165/1000
5460/5460 [==============================] - 0s 67us/sample - loss: 0.2500 - acc: 0.5048
Epoch 166/1000
5460/5460 [==============================] - 0s 68us/sample - loss: 0.2500 - acc: 0.5075
Epoch 167/1000
5460/5460 [==============================] - 0s 67us/sample - loss: 0.2500 - acc: 0.5035
Epoch 168/1000
5460/5460 [==============================] - 0s 67us/sample - lo

5460/5460 [==============================] - 0s 69us/sample - loss: 0.2500 - acc: 0.5013
Epoch 316/1000
5460/5460 [==============================] - 0s 69us/sample - loss: 0.2500 - acc: 0.5007
Epoch 317/1000
5460/5460 [==============================] - 0s 67us/sample - loss: 0.2500 - acc: 0.5000
Epoch 318/1000
5460/5460 [==============================] - 0s 70us/sample - loss: 0.2500 - acc: 0.5071
Epoch 319/1000
5460/5460 [==============================] - 0s 67us/sample - loss: 0.2500 - acc: 0.5027
Epoch 320/1000
5460/5460 [==============================] - 0s 67us/sample - loss: 0.2500 - acc: 0.4916
Epoch 321/1000
5460/5460 [==============================] - 0s 68us/sample - loss: 0.2500 - acc: 0.4921
Epoch 322/1000
5460/5460 [==============================] - 0s 69us/sample - loss: 0.2500 - acc: 0.4998
Epoch 323/1000
5460/5460 [==============================] - 0s 66us/sample - loss: 0.2500 - acc: 0.4952
Epoch 324/1000
5460/5460 [==============================] - 0s 65us/sample - lo

5460/5460 [==============================] - 0s 69us/sample - loss: 0.2500 - acc: 0.4971
Epoch 472/1000
5460/5460 [==============================] - 0s 66us/sample - loss: 0.2500 - acc: 0.4852
Epoch 473/1000
5460/5460 [==============================] - 0s 69us/sample - loss: 0.2500 - acc: 0.5024
Epoch 474/1000
5460/5460 [==============================] - 0s 66us/sample - loss: 0.2500 - acc: 0.5005
Epoch 475/1000
5460/5460 [==============================] - 0s 67us/sample - loss: 0.2500 - acc: 0.5068
Epoch 476/1000
5460/5460 [==============================] - 0s 67us/sample - loss: 0.2500 - acc: 0.4930
Epoch 477/1000
5460/5460 [==============================] - 0s 67us/sample - loss: 0.2500 - acc: 0.4930
Epoch 478/1000
5460/5460 [==============================] - 0s 67us/sample - loss: 0.2500 - acc: 0.4971
Epoch 479/1000
5460/5460 [==============================] - 0s 67us/sample - loss: 0.2500 - acc: 0.4958
Epoch 480/1000
5460/5460 [==============================] - 0s 66us/sample - lo

5460/5460 [==============================] - 0s 69us/sample - loss: 0.2500 - acc: 0.4958
Epoch 628/1000
5460/5460 [==============================] - 0s 68us/sample - loss: 0.2500 - acc: 0.5062
Epoch 629/1000
5460/5460 [==============================] - 0s 67us/sample - loss: 0.2500 - acc: 0.4916
Epoch 630/1000
5460/5460 [==============================] - 0s 67us/sample - loss: 0.2500 - acc: 0.4954
Epoch 631/1000
5460/5460 [==============================] - 0s 68us/sample - loss: 0.2500 - acc: 0.4987
Epoch 632/1000
5460/5460 [==============================] - 0s 67us/sample - loss: 0.2500 - acc: 0.4985
Epoch 633/1000
5460/5460 [==============================] - 0s 67us/sample - loss: 0.2500 - acc: 0.4949
Epoch 634/1000
5460/5460 [==============================] - 0s 67us/sample - loss: 0.2500 - acc: 0.5081
Epoch 635/1000
5460/5460 [==============================] - 0s 67us/sample - loss: 0.2500 - acc: 0.5062
Epoch 636/1000
5460/5460 [==============================] - 0s 66us/sample - lo

5460/5460 [==============================] - 0s 69us/sample - loss: 0.2500 - acc: 0.4967
Epoch 784/1000
5460/5460 [==============================] - 0s 68us/sample - loss: 0.2500 - acc: 0.4993
Epoch 785/1000
5460/5460 [==============================] - 0s 67us/sample - loss: 0.2500 - acc: 0.4954
Epoch 786/1000
5460/5460 [==============================] - 0s 67us/sample - loss: 0.2500 - acc: 0.4940
Epoch 787/1000
5460/5460 [==============================] - 0s 68us/sample - loss: 0.2500 - acc: 0.4980
Epoch 788/1000
5460/5460 [==============================] - 0s 67us/sample - loss: 0.2500 - acc: 0.5075
Epoch 789/1000
5460/5460 [==============================] - 0s 67us/sample - loss: 0.2500 - acc: 0.4875
Epoch 790/1000
5460/5460 [==============================] - 0s 67us/sample - loss: 0.2500 - acc: 0.5042
Epoch 791/1000
5460/5460 [==============================] - 0s 67us/sample - loss: 0.2500 - acc: 0.5037
Epoch 792/1000
5460/5460 [==============================] - 0s 67us/sample - lo

5460/5460 [==============================] - 0s 65us/sample - loss: 0.2500 - acc: 0.4925
Epoch 940/1000
5460/5460 [==============================] - 0s 63us/sample - loss: 0.2500 - acc: 0.5059
Epoch 941/1000
5460/5460 [==============================] - 0s 63us/sample - loss: 0.2500 - acc: 0.5016
Epoch 942/1000
5460/5460 [==============================] - 0s 64us/sample - loss: 0.2500 - acc: 0.5011
Epoch 943/1000
5460/5460 [==============================] - 0s 63us/sample - loss: 0.2500 - acc: 0.5002
Epoch 944/1000
5460/5460 [==============================] - 0s 63us/sample - loss: 0.2500 - acc: 0.5031
Epoch 945/1000
5460/5460 [==============================] - 0s 64us/sample - loss: 0.2500 - acc: 0.4905
Epoch 946/1000
5460/5460 [==============================] - 0s 64us/sample - loss: 0.2500 - acc: 0.4998
Epoch 947/1000
5460/5460 [==============================] - 0s 64us/sample - loss: 0.2500 - acc: 0.4993
Epoch 948/1000
5460/5460 [==============================] - 0s 63us/sample - lo